In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, LeakyReLU
from keras.losses import mse, binary_crossentropy
from keras.utils import np_utils, plot_model
from keras import backend as K
from keras import optimizers
from keras import regularizers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/train_labels.csv
/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/test_data.npy


In [2]:
train_data = np.load('/kaggle/input/audio-binary-classification/train_data.npy');
test_data = np.load('/kaggle/input/audio-binary-classification/test_data.npy');


print("inputs loaded")

inputs loaded


In [3]:
#Scale and Reshape
train_data = train_data.astype('float32');
test_data = test_data.astype('float32');

#Normalize Data
#train_data /= np.amax(test_data);
#test_data /= np.amax(test_data);

print(train_data.shape);

train_data = train_data.reshape(-1,44100,1);
test_data = test_data.reshape(-1,44100,1);

input_shape = train_data.shape;
print(input_shape);

(3999, 44100)
(3999, 44100, 1)


In [4]:
#Look at CSV and store as array
train_labels = pd.read_csv("/kaggle/input/audio-binary-classification/train_labels.csv");
train_labels = train_labels.values;
train_labels = train_labels[:,1];
print(train_labels[:10]);
print(train_labels.shape)

[0 0 0 1 0 0 1 0 1 0]
(3999,)


In [5]:
#Modify data
x_data = np.zeros( (train_data.shape[0] * 2 + 250, train_data.shape[1],1) );
y_data = np.zeros( (train_labels.shape[0] * 2 + 250))

x_data[:3999,:,0] = train_data[:3999,:,0];
y_data[:3999] = train_labels[:3999];

for a in range(train_data.shape[0]):
    s = np.random.normal(1,.02,44100);
    x_data[a+3999,:,0] = np.multiply(x_data[a,:,0],s);
    y_data[a+3999] = train_labels[a];
        
b = 0;

for a in range(train_data.shape[0]):
    if(train_labels[a] == 1):
        s = np.random.normal(1,.02,44100);
        x_data[2*3999+b,:,0] = np.multiply(x_data[a,:,0],s);
        y_data[2*3999+b] = train_labels[a];
        b += 1;
    if(b >= 250):
        break;
        
print("data made noisey")
        

data made noisey


In [6]:
print(train_data[1,:,0]);
print(x_data[4000,:,0])

[0.00110629 0.00221578 0.00401346 ... 0.02375466 0.01968718 0.01686463]
[0.00109889 0.00225888 0.00400161 ... 0.02358773 0.01977906 0.01693984]


In [7]:
# create convolution neural network
model = Sequential()
model.add(Conv1D(64, kernel_size=(10),
                 input_shape=(44100,1)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.33))

model.add(Conv1D(64, (8)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.33))

model.add(Conv1D(64, (5)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Dropout(0.33))

model.add(Flatten())
model.add(Dense(80))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(1, activation='sigmoid'))

model.summary();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 44091, 64)         704       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 44091, 64)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11022, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 11022, 64)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11015, 64)         32832     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 11015, 64)         0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1835, 64)          0         
__________

In [8]:
#Compile model
sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer='sgd',
              metrics=['accuracy'])

In [9]:
history = model.fit(x_data,y_data,
          batch_size=30,epochs=350,verbose=1)

Epoch 1/350
8248/8248 [==============================] - 18s 2ms/step - loss: 0.1270 - acc: 0.8491
Epoch 2/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0990 - acc: 0.8674
Epoch 3/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0815 - acc: 0.8977
Epoch 4/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0745 - acc: 0.9073
Epoch 5/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0717 - acc: 0.9085
Epoch 6/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0705 - acc: 0.9089
Epoch 7/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0688 - acc: 0.9121
Epoch 8/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0674 - acc: 0.9126
Epoch 9/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0662 - acc: 0.9132
Epoch 10/350
8248/8248 [==============================] - 15s 2ms/step - loss: 0.0649 - acc: 0.9139
Epoch 11/

In [10]:
Y_pred = model.predict(test_data);
print(Y_pred)
df_pred = pd.DataFrame(Y_pred);
df_pred.index.name = "Id"
df_pred = pd.DataFrame.rename(df_pred,columns={0:"Label"})
print(df_pred);
df_pred.to_csv("output.csv")

[[9.2387199e-07]
 [1.0816157e-03]
 [6.9976151e-03]
 ...
 [2.0861626e-07]
 [1.5318990e-03]
 [9.7423065e-01]]
             Label
Id                
0     9.238720e-07
1     1.081616e-03
2     6.997615e-03
3     0.000000e+00
4     3.576279e-07
...            ...
3992  9.048820e-03
3993  2.980232e-08
3994  2.086163e-07
3995  1.531899e-03
3996  9.742306e-01

[3997 rows x 1 columns]
